# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. These are combined pluvial and fluvial (undefended) flood layers whereby whichever flood level was higher was retained.
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box, Polygon
from shapely.ops import unary_union, linemerge, transform
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
from GOSTnets import load_osm as losm
import importlib

In [7]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [8]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [9]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Prepare OSM driving network.
Travel measured in length (meters).

#### Ensure all targeted roads are changed to tertiary. 

In [10]:
flood50 = gpd.read_file("C:/Users/wb527163/GEO-Cdrive-Grace/SEN-Cdrive/scratch.gdb", layer="PFU_1in50")
flood50.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2459356 entries, 0 to 2459355
Data columns (total 4 columns):
 #   Column        Dtype   
---  ------        -----   
 0   PFU_1in50     float64 
 1   Shape_Length  float64 
 2   Shape_Area    float64 
 3   geometry      geometry
dtypes: float64(3), geometry(1)
memory usage: 75.1 MB


In [11]:
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime_post-project.pickle")

In [12]:
edges = gn.edge_gdf_from_graph(gTime)

In [13]:
nodes = os.path.join(out_pth, "gTime_node_post-project.csv")
nodes = pd.read_csv(nodes)
print(nodes.info())
print(edges.info())

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871182 entries, 0 to 2871181
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   node_ID       int64  
 2   ref           object 
 3   highway       object 
 4   x             float64
 5   Unnamed: 0.1  int64  
 6   y             float64
 7   geometry      object 
dtypes: float64(2), int64(3), object(3)
memory usage: 175.2+ MB
None
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6203383 entries, 0 to 6203382
Data columns (total 23 columns):
 #   Column        Dtype   
---  ------        -----   
 0   stnode        int64   
 1   endnode       int64   
 2   access        object  
 3   mode          object  
 4   length        float64 
 5   Unnamed: 0    int64   
 6   junction      object  
 7   lanes         float64 
 8   highway       object  
 9   time          float64 
 10  area          object  
 11  landuse       object  
 12  oneway        bool    
 13  bri

In [14]:
# Spatial join should be on projected GDFs.
edges = edges.to_crs("EPSG:31028")
flood50 = flood50.to_crs("EPSG:31028")
edges.crs == flood50.crs

True

In [15]:
floodjoin = gpd.sjoin_nearest(edges, flood50, how="left", max_distance=3) 
print(floodjoin.info())
print(floodjoin)

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6508966 entries, 0 to 6203382
Data columns (total 27 columns):
 #   Column        Dtype   
---  ------        -----   
 0   stnode        int64   
 1   endnode       int64   
 2   access        object  
 3   mode          object  
 4   length        float64 
 5   Unnamed: 0    int64   
 6   junction      object  
 7   lanes         float64 
 8   highway       object  
 9   time          float64 
 10  area          object  
 11  landuse       object  
 12  oneway        bool    
 13  bridge        object  
 14  ref           object  
 15  service       object  
 16  Unnamed: 0.1  int64   
 17  tunnel        object  
 18  name          object  
 19  osmid         int64   
 20  width         object  
 21  maxspeed      object  
 22  geometry      geometry
 23  index_right   float64 
 24  PFU_1in50     float64 
 25  Shape_Length  float64 
 26  Shape_Area    float64 
dtypes: bool(1), float64(7), geometry(1), int64(5), object(13)
memo

In [16]:
# How many nodes experienced flooding?
pc_flooded = floodjoin["PFU_1in50"].count() / len(floodjoin) * 100

print("No flood crossing at node:", floodjoin["PFU_1in50"].isnull().sum(), "locations", end="\n\n")
print("Flood crossing at node:", floodjoin["PFU_1in50"].count(), "locations", end="\n\n")
print("\nPercent flooded:", pc_flooded, "percent", "out of", len(floodjoin), "possible locations")

No flood crossing at node: 5745481 locations

Flood crossing at node: 763485 locations


Percent flooded: 11.729743249542247 percent out of 6508966 possible locations


In [17]:
floodjoin = floodjoin[['stnode', 'endnode', 'time', 'length', 'highway', 'osmid', 'geometry', 'PFU_1in50']]
floodjoin = floodjoin.to_crs("EPSG:4326")
floodjoin.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6508966 entries, 0 to 6203382
Data columns (total 8 columns):
 #   Column     Dtype   
---  ------     -----   
 0   stnode     int64   
 1   endnode    int64   
 2   time       float64 
 3   length     float64 
 4   highway    object  
 5   osmid      int64   
 6   geometry   geometry
 7   PFU_1in50  float64 
dtypes: float64(3), geometry(1), int64(3), object(1)
memory usage: 446.9+ MB


In [18]:
# Fewer errors farther down when using dataframe instead of gdf
floodjoin = pd.DataFrame(floodjoin)

In [19]:
# Save progress.
floodjoin.to_csv(os.path.join(out_pth, 'gTime_edge_post-project_flood50.csv'))

### Create speed penalties.
Note: Flood depths are in centimeters. FATHOM uses meters, but conversion process to vector required some finessing. 

In [20]:
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'edges', 'exit', 'fiona', 'flood50', 'floodjoin', 'gTime', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'nodes', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']


In [21]:
del flood50, gTime, edges, floodjoin
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'exit', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'nodes', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']


In [22]:
# Alternatively, restart the kernel and reload.
floodjoin = os.path.join(out_pth, "gTime_edge_post-project_flood50.csv")
floodjoin = pd.read_csv(floodjoin)

In [23]:
# Give a depth to the nodes that don't cross a flood point. 
floodjoin.loc[floodjoin['PFU_1in50'].isnull(), 'PFU_1in50'] = -1
floodjoin["t50"] = 1 # This is the penalty column.
floodjoin.loc[floodjoin['PFU_1in50'] <= 5, 't50'] = 1 # Where no flood crosses, keep the default value (no penalty).
floodjoin.loc[(floodjoin['PFU_1in50'] > 5) & (floodjoin['PFU_1in50'] <= 10), 't50'] = 1.25 # Between 10-30cm flooding, increase travel time by 1.25.
floodjoin.loc[(floodjoin['PFU_1in50'] > 10) & (floodjoin['PFU_1in50'] <= 15), 't50'] = 2
floodjoin.loc[(floodjoin['PFU_1in50'] > 15) & (floodjoin['PFU_1in50'] <= 40), 't50'] = 5
floodjoin.loc[(floodjoin['PFU_1in50'] > 40), 't50'] = 9999
floodjoin

,Unnamed: 0,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50,t50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,1.0
3,3,5217543379,358284990,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.0
4,4,5217543379,5329928981,1.232928,17.124,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
6508961,6203378,9165665160,9165665161,10.636000,13.295,path,992024621,LINESTRING (-17.48487070737049 14.745767002669...,-1.0,1.0
6508962,6203379,9165665160,9165665159,10.385600,12.982,path,992024621,LINESTRING (-17.48487070737049 14.745767002669...,-1.0,1.0
6508963,6203380,9165665161,9165665162,4.380800,5.476,path,992024621,LINESTRING (-17.48475000737059 14.745741102669...,-1.0,1.0
6508964,6203381,9165665161,9165665160,10.636000,13.295,path,992024621,LINESTRING (-17.48475000737059 14.745741102669...,-1.0,1.0


In [24]:
targeted = os.path.join(pth, "targeted_roads_osmid.csv")
targeted = pd.read_csv(targeted)
print(targeted.info(), end="\n\n")
targeted = targeted[['id']]
targeted['id'] = targeted['id'].str.replace(r'\D', '').astype(int)
# Do NOT remove Mboro this time. It was not originally a track, but it will be rehabilitated. 
# targeted = targeted.drop([targeted.index[49]]) # Mboro-Diogo stays the same.
target_list = list(targeted.id)
print(target_list)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          59 non-null     object 
 1   @id         59 non-null     object 
 2   highway     59 non-null     object 
 3   name        5 non-null      object 
 4   oneway      26 non-null     object 
 5   smoothness  2 non-null      object 
 6   surface     48 non-null     object 
 7   access      0 non-null      float64
 8   note        0 non-null      float64
 9   source      0 non-null      float64
 10  tracktype   17 non-null     object 
 11  bridge      0 non-null      float64
 12  layer       0 non-null      float64
 13  embankment  0 non-null      float64
 14  ref         3 non-null      object 
 15  lanes       21 non-null     float64
 16  maxspeed    7 non-null      float64
 17  lit         0 non-null      float64
 18  ford        0 non-null      float64
 19  fixme       0 non-null      flo

C:\Users\wb527163\AppData\Local\Temp/ipykernel_22832/2148227697.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  targeted['id'] = targeted['id'].str.replace(r'\D', '').astype(int)


In [25]:
# Turn the penalty column into a flood-affected time column.
floodjoin['t50'] = floodjoin['t50'] * floodjoin['time']
# The project will implement flood protection that prevents isolation. Make sure that's reflected in this simulation.
floodjoin.loc[floodjoin['osmid'].isin(target_list), 
          't50'] = floodjoin['time'] 
floodjoin

,Unnamed: 0,stnode,endnode,time,length,highway,osmid,geometry,PFU_1in50,t50
0,0,358284990,5217543379,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,2.385144
1,1,358284990,1888282175,0.769920,12.832,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,0.769920
2,2,358284990,5329792467,2.926860,48.781,tertiary,178482063,LINESTRING (-12.323470812149301 12.38118950295...,-1.0,2.926860
3,3,5217543379,358284990,2.385144,33.127,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,2.385144
4,4,5217543379,5329928981,1.232928,17.124,unclassified,59618174,LINESTRING (-12.323682912149085 12.38140360295...,-1.0,1.232928
...,...,...,...,...,...,...,...,...,...,...
6508961,6203378,9165665160,9165665161,10.636000,13.295,path,992024621,LINESTRING (-17.48487070737049 14.745767002669...,-1.0,10.636000
6508962,6203379,9165665160,9165665159,10.385600,12.982,path,992024621,LINESTRING (-17.48487070737049 14.745767002669...,-1.0,10.385600
6508963,6203380,9165665161,9165665162,4.380800,5.476,path,992024621,LINESTRING (-17.48475000737059 14.745741102669...,-1.0,4.380800
6508964,6203381,9165665161,9165665160,10.636000,13.295,path,992024621,LINESTRING (-17.48475000737059 14.745741102669...,-1.0,10.636000


In [26]:
# Save progress.
floodjoin.to_csv(os.path.join(out_pth, 'gTime_edge_post-project_flood50.csv'))

In [27]:
print("Travel time without inclement weather\n")
print("Mean:", floodjoin['time'].mean(), end="\n")
print("Median:", floodjoin['time'].median(), end='\n\n')
print("\n\nDuring 1 in 50-year flood conditions\n")
print("Mean:", floodjoin['t50'].mean(), end="\n") 
print("Median:", floodjoin['t50'].median()) 

Travel time without inclement weather

Mean: 7.447111958077438
Median: 3.282192



During 1 in 50-year flood conditions

Mean: 8436.681632176585
Median: 3.5952


### Convert back to graph object.

In [28]:
# Converting back to graph can cause memory errors. Suggested to restart the kernel and reload the nodes and revised edges at this point.
del floodjoin, nodes, targeted
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_25', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'exit', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'target_list', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']


In [29]:
nodes = os.path.join(out_pth, "gTime_node_post-project.csv")
nodes = pd.read_csv(nodes)
floodjoin = os.path.join(out_pth, "gTime_edge_post-project_flood50.csv")
floodjoin = pd.read_csv(floodjoin)
print(nodes.info())
print(floodjoin.info())

C:\Users\wb527163\.conda\envs\geo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871182 entries, 0 to 2871181
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   node_ID       int64  
 2   ref           object 
 3   highway       object 
 4   x             float64
 5   Unnamed: 0.1  int64  
 6   y             float64
 7   geometry      object 
dtypes: float64(2), int64(3), object(3)
memory usage: 175.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6508966 entries, 0 to 6508965
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   Unnamed: 0.1  int64  
 2   stnode        int64  
 3   endnode       int64  
 4   time          float64
 5   length        float64
 6   highway       object 
 7   osmid         int64  
 8   geometry      object 
 9   PFU_1in50     float64
 10  t50           float64
dtypes: float64(4), int64(5), object(2)
memory usage: 546.3+ MB
None


In [30]:
print('start: %s\n' % time.ctime())
gTime = gn.edges_and_nodes_gdf_to_graph(nodes, floodjoin, node_tag='node_ID', u_tag='stnode', v_tag='endnode', geometry_tag='geometry')
gn.example_edge(gTime, 10)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Sun Jan  9 17:34:40 2022

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194FF43A3A0>, 'Unnamed: 0': 0, 'Unnamed: 0.1': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in50': -1.0, 't50': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194FF43AA00>, 'Unnamed: 0': 1, 'Unnamed: 0.1': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194FF43AB80>, 'Unnamed: 0': 2, 'Unnamed: 0.1': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194FF43A940>, 'Unnamed: 0': 3, 'Unnamed: 0.1': 3, 'time': 2.38514400000000

In [31]:
print('start: %s\n' % time.ctime())
gn.save(gTime, 'gTime_post-project_flood50', out_pth, edges = False, nodes = False)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Sun Jan  9 17:50:22 2022


end: Sun Jan  9 17:53:47 2022

--- processing complete


### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

Already measured distance from origin/destination to nearest node in pre-project script.

In [32]:
#%% If starting new session, reload graph from file
del gTime, floodjoin, nodes
print(dir())
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime_post-project_flood50.pickle")
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_25', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'exit', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'loads', 'losm', 'np', 'nx', 'os', 'out_pth', 'ox', 'pc_flooded', 'pd', 'peartree', 'plt', 'pth', 'quit', 'rt', 'shapely', 'subprocess', 'sys', 'target_list', 'team_pth', 'time', 'transform', 'unary_union', 'wkt']
['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_25', '_7', '_8', '_9', '__

In [33]:
HDurban_snap = os.path.join(out_pth, "HDurban_snap.csv")
HDurban_snap = pd.read_csv(HDurban_snap)
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv")
hamlet_snap = pd.read_csv(hamlet_snap)

In [34]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)

In [35]:
HDurban_snap

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,AREA_GEO,SUM,Total_Driv,WP_dens,urb,hd_urb,Urb_class,Shape_Leng,Shape_Area,geometry,NN,NN_dist
0,0,1,28PCU6189_01,bua,27.151434,1.860466e+05,0.000000,6852.183502,1,1,2,0.671746,2.258828e-03,POINT (-16.27318626010657 12.562367084144856),6058226279,2.216606
1,1,8,28PEV0625_01,bua,25.799549,1.064198e+05,0.000000,4124.872441,1,1,2,0.718941,2.149259e-03,POINT (-14.936187613172205 12.892160925397892),6029307183,45.594167
2,2,37,28PBA8597_01,bua,81.242354,3.382677e+05,0.000000,4163.686114,1,1,2,2.303639,6.813610e-03,POINT (-16.987028969206293 14.445279143289168),4998093094,55.383696
3,3,47,28PCA8365_01,bua,42.134069,2.755403e+05,0.000000,6539.609258,1,1,2,1.070058,3.528274e-03,POINT (-16.076128475299285 14.160382012387636),2201506815,55.754910
4,4,56,28PDA4058_01,bua,11.123881,5.186398e+04,0.000000,4662.399756,1,1,2,0.298223,9.311211e-04,POINT (-15.547596052646174 14.100419699131665),4321656809,61.727308
5,5,75,28PBB9234_01,bua,53.453391,2.911360e+05,0.000000,5446.539082,1,1,2,1.468618,4.489114e-03,POINT (-16.926149059908756 14.780621967102865),1697006012,23.047883
6,6,76,28PBB4632_01,bua,226.850348,3.637718e+06,0.000000,16035.762110,1,1,2,4.771359,1.905124e-02,POINT (-17.35638706164378 14.751962825615575),1901689169,5.826006
7,7,80,28PCB0453_01,bua,19.580942,8.712281e+04,0.000000,4449.367491,1,1,2,0.611345,1.645460e-03,POINT (-16.81720508223022 14.94580992504745),6032060028,88.156089
8,8,84,28PCB6720_01,bua,23.634834,1.342721e+05,0.000000,5681.109779,1,1,2,0.689064,1.983306e-03,POINT (-16.227786747221682 14.653718853747844),6040927878,41.930884
9,9,98,28PDB0443_01,bua,196.802191,9.490037e+05,0.000000,2575.818720,1,1,2,3.094788,1.653195e-02,POINT (-15.887758561636817 14.860301673230607),3449495495,28.431974


In [36]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
list_hamlet = list(hamlet_snap.NN.unique())
origins = list_hamlet

In [37]:
list_ag = list(ag_snap.NN.unique())
originslist = list_hamlet + list_ag
origins = list(set(originslist))

In [38]:
dests = list(HDurban_snap.NN.unique()) 

In [39]:
len(origins) # 985720 unique nearest nodes.

985720

In [40]:
len(dests) # 58 unique nearest nodes. 

58

In [41]:
gn.example_edge(gTime,10)

(358284990, 5217543379, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194977EDBB0>, 'Unnamed: 0': 0, 'Unnamed: 0.1': 0, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 'unclassified', 'osmid': 59618174, 'PFU_1in50': -1.0, 't50': 2.3851440000000004})
(358284990, 1888282175, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194977ED4F0>, 'Unnamed: 0': 1, 'Unnamed: 0.1': 1, 'time': 0.76992, 'length': 12.832, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 0.76992})
(358284990, 5329792467, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194977ED3A0>, 'Unnamed: 0': 2, 'Unnamed: 0.1': 2, 'time': 2.92686, 'length': 48.781, 'highway': 'tertiary', 'osmid': 178482063, 'PFU_1in50': -1.0, 't50': 2.92686})
(5217543379, 358284990, {'geometry': <shapely.geometry.linestring.LineString object at 0x00000194977EDF40>, 'Unnamed: 0': 3, 'Unnamed: 0.1': 3, 'time': 2.3851440000000004, 'length': 33.127, 'highway': 

calculate_OD won't run if any of the edge times are null or zero. If there are any hits, reassign time values for those edges to a very small time. A more efficient way to do this would be to simplify junctions with the clean_network() tool, but that was throwing errors.

In [42]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [43]:
print('start: %s\n' % time.ctime())
OD = gn.calculate_OD(gTime, origins, dests, fail_value, weight = 't50')
# Takes a few minutes.
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Sun Jan  9 17:59:10 2022


end: Sun Jan  9 18:36:11 2022

--- processing complete


In [44]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [45]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985720 entries, 3331684166 to 3706826877
Data columns (total 58 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   6058226279  985720 non-null  float64
 1   6029307183  985720 non-null  float64
 2   4998093094  985720 non-null  float64
 3   2201506815  985720 non-null  float64
 4   4321656809  985720 non-null  float64
 5   1697006012  985720 non-null  float64
 6   1901689169  985720 non-null  float64
 7   6032060028  985720 non-null  float64
 8   6040927878  985720 non-null  float64
 9   3449495495  985720 non-null  float64
 10  3990543961  985720 non-null  float64
 11  8972391475  985720 non-null  float64
 12  3418418812  985720 non-null  float64
 13  1983641803  985720 non-null  float64
 14  6014451367  985720 non-null  float64
 15  6027163276  985720 non-null  float64
 16  2833577858  985720 non-null  float64
 17  7321088861  985720 non-null  float64
 18  6045659373  985720 non-null  fl

In [46]:
OD_df.tail()

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
8925478824,61632.315005,221724.009077,6294.513822,10817.165367,31985.753411,4119.039466,6514.333964,2932.849334,118766.313972,5300.161107,...,9043.084400,10281.605269,141901.333357,141883.734595,6547.461643,6106.681600,6074.985768,6069.733378,5435.576714,5485.793684
3706826868,301627.683774,461719.377846,301544.344971,293245.207878,263796.470394,299478.115069,301958.551698,298380.699995,401194.356483,290514.817687,...,303471.094708,304709.615577,381896.702126,381879.103364,301797.292792,301550.899333,301519.203501,301513.951111,290723.625097,291135.400807
3706826871,232191.891657,392283.585729,232108.552854,223809.415761,194360.678277,230042.322952,232522.759581,228944.907878,331758.564366,221079.025571,...,234035.302591,235273.823460,312460.910008,312443.311247,232361.500676,232115.107217,232083.411384,232078.158995,221287.832981,221699.608691
3706826876,231906.653046,391998.347118,231823.314243,223524.177150,194075.439666,229757.084341,232237.520970,228659.669267,331473.325755,220793.786959,...,233750.063980,234988.584849,312175.671398,312158.072636,232076.262064,231829.868605,231798.172773,231792.920383,221002.594369,221414.370079
3706826877,232207.006905,392298.700977,232123.668102,223824.531009,194375.793525,230057.438200,232537.874829,228960.023126,331773.679614,221094.140819,...,234050.417839,235288.938708,312476.025257,312458.426495,232376.615924,232130.222464,232098.526632,232093.274242,221302.948229,221714.723939


In [47]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(out_pth, 'OD_allorigins_post-project_flood50.csv'))
OD_min

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3331684166,997.018955,3665.213856,288.418746,11.536460,502.909595,253.981581,295.322191,235.691330,1869.250269,104.593291,...,320.531242,341.173256,2334.835928,2334.542615,292.634543,288.527985,287.999722,287.912182,108.073415,114.936343
3571449893,520.662115,3042.737410,750.029663,611.710712,257.309284,715.592498,756.933109,697.302247,2410.863189,566.204209,...,782.142159,802.784174,1712.359481,1712.066169,754.245460,750.138903,749.610639,749.523099,569.684332,576.547261
3571449966,521.049027,3043.124322,750.416575,612.097623,257.696195,715.979410,757.320020,697.689159,2411.250100,566.591120,...,782.529071,803.171085,1712.746393,1712.453080,754.632372,750.525814,749.997550,749.910011,570.071244,576.934172
3405774993,396.351017,3178.487893,739.660540,601.341588,246.940160,705.223375,746.563985,686.933123,2400.494065,555.835085,...,771.773035,792.415050,1848.109964,1847.816651,743.876337,739.769779,739.241515,739.153975,559.315208,566.178137
3405774994,395.950268,3178.087144,739.259791,600.940839,246.539411,704.822626,746.163236,686.532374,2400.093316,555.434336,...,771.372286,792.014301,1847.709215,1847.415902,743.475588,739.369030,738.840766,738.753226,558.914459,565.777388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8925478824,1027.205250,3695.400151,104.908564,180.286089,533.095890,68.650658,108.572233,48.880822,1979.438566,88.336018,...,150.718073,171.360088,2365.022223,2364.728910,109.124361,101.778027,101.249763,101.162223,90.592945,91.429895
3706826868,5027.128063,7695.322964,5025.739083,4887.420131,4396.607840,4991.301918,5032.642528,4973.011667,6686.572608,4841.913628,...,5057.851578,5078.493593,6364.945035,6364.651723,5029.954880,5025.848322,5025.320058,5025.232519,4845.393752,4852.256680
3706826871,3869.864861,6538.059762,3868.475881,3730.156929,3239.344638,3834.038716,3875.379326,3815.748465,5529.309406,3684.650426,...,3900.588377,3921.230391,5207.681833,5207.388521,3872.691678,3868.585120,3868.056856,3867.969317,3688.130550,3694.993478
3706826876,3865.110884,6533.305785,3863.721904,3725.402953,3234.590661,3829.284739,3870.625349,3810.994488,5524.555429,3679.896449,...,3895.834400,3916.476414,5202.927857,5202.634544,3867.937701,3863.831143,3863.302880,3863.215340,3683.376573,3690.239501


In [48]:
print(OD_min.isna().sum())

6058226279    3
6029307183    3
4998093094    3
2201506815    3
4321656809    3
1697006012    3
1901689169    3
6032060028    3
6040927878    3
3449495495    3
3990543961    3
8972391475    3
3418418812    3
1983641803    3
6014451367    3
6027163276    3
2833577858    3
7321088861    3
6045659373    3
5528866190    3
6027939517    3
6031093257    3
1883155712    3
6026417971    3
6031245697    3
7493947593    3
1859359090    3
6053816462    3
6027530676    3
2988435647    3
7838754747    3
6053825179    3
2988435792    3
3036877534    3
6056472223    3
7460241128    3
6028595154    3
4053739078    3
4071261250    3
6014815775    3
6024430122    3
8229317581    3
6027834438    3
6024894515    3
6024894530    3
6024894513    3
8169321141    3
8200304626    3
1968458114    3
9359670710    3
6021177135    3
6027615161    3
6027276892    3
6041228287    3
5536661253    3
7357630367    3
8178147277    3
6026834850    3
dtype: int64


In [49]:
# Create origin-specific matrix and save to file.
OD_hamlet = OD_df.loc[list_hamlet,:]
OD_hamlet = OD_hamlet[OD_hamlet < fail_value] / 60 
OD_hamlet.to_csv(os.path.join(out_pth, 'OD_hamlet_post-project_flood50.csv'))
OD_hamlet

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
7761872870,7555.041976,11114.143037,8675.315684,8536.996732,8182.595304,8640.878519,8682.219129,8622.588267,10336.149209,8491.490229,...,8707.428179,8728.070194,9783.765108,9783.471795,8679.531481,8675.424923,8674.896659,8674.809119,8494.970352,8501.833281
7761872869,7555.023417,11114.124477,8675.297124,8536.978173,8182.576744,8640.859959,8682.200570,8622.569708,10336.130649,8491.471669,...,8707.409620,8728.051634,9783.746548,9783.453236,8679.512921,8675.406364,8674.878100,8674.790560,8494.951793,8501.814721
6442044321,7554.013585,11113.114645,8674.287292,8535.968341,8181.566912,8639.850127,8681.190738,8621.559876,10335.120817,8490.461837,...,8706.399788,8727.041802,9782.736716,9782.443404,8678.503089,8674.396532,8673.868268,8673.780728,8493.941961,8500.804889
2142496418,7555.467107,11114.568167,8675.740814,8537.421863,8183.020434,8641.303649,8682.644260,8623.013398,10336.574339,8491.915359,...,8707.853310,8728.495324,9784.190238,9783.896926,8679.956611,8675.850054,8675.321790,8675.234250,8495.395483,8502.258411
2142496429,7555.849481,11114.950541,8676.123188,8537.804237,8183.402808,8641.686023,8683.026634,8623.395772,10336.956713,8492.297733,...,8708.235684,8728.877698,9784.572612,9784.279300,8680.338985,8676.232428,8675.704164,8675.616624,8495.777857,8502.640785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9208004175,402592.030991,405260.225892,401657.732281,401745.111830,402097.921631,401602.316288,401642.237863,401583.581214,403544.264307,401653.161759,...,401498.200522,401426.662175,403929.847963,403929.554651,401661.948078,401635.443657,401634.915393,401634.827853,401655.418686,401656.255636
3884280043,1900.785584,4568.980485,1151.720971,1053.866424,1406.676224,1114.461377,1154.382952,1095.693230,2853.018900,962.437282,...,1001.608998,1075.735769,3238.602557,3238.309244,1155.936768,1147.588746,1147.060482,1146.972943,971.630082,964.146135
3884280001,1893.923357,4562.118258,1144.858744,1047.004197,1399.813997,1107.599150,1147.520725,1088.831003,2846.156673,955.575055,...,994.746771,1068.873542,3231.740330,3231.447017,1149.074541,1140.726519,1140.198255,1140.110716,964.767855,957.283908
8592243457,1869.716935,4537.911836,1120.652322,1022.797774,1375.607575,1083.392728,1123.314303,1064.624581,2821.950251,931.368633,...,970.540349,1044.667120,3207.533907,3207.240595,1124.868119,1116.520097,1115.991833,1115.904293,940.561432,933.077486


In [50]:
print(OD_hamlet.isna().sum())

6058226279    1
6029307183    1
4998093094    1
2201506815    1
4321656809    1
1697006012    1
1901689169    1
6032060028    1
6040927878    1
3449495495    1
3990543961    1
8972391475    1
3418418812    1
1983641803    1
6014451367    1
6027163276    1
2833577858    1
7321088861    1
6045659373    1
5528866190    1
6027939517    1
6031093257    1
1883155712    1
6026417971    1
6031245697    1
7493947593    1
1859359090    1
6053816462    1
6027530676    1
2988435647    1
7838754747    1
6053825179    1
2988435792    1
3036877534    1
6056472223    1
7460241128    1
6028595154    1
4053739078    1
4071261250    1
6014815775    1
6024430122    1
8229317581    1
6027834438    1
6024894515    1
6024894530    1
6024894513    1
8169321141    1
8200304626    1
1968458114    1
9359670710    1
6021177135    1
6027615161    1
6027276892    1
6041228287    1
5536661253    1
7357630367    1
8178147277    1
6026834850    1
dtype: int64


In [51]:
# Create origin-specific matrix and save to file.
OD_ag = OD_df.loc[list_ag,:]
OD_ag = OD_ag[OD_ag < fail_value] / 60 
OD_ag.to_csv(os.path.join(out_pth, 'OD_ag_post-project_flood50.csv'))
OD_ag

,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,3449495495,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
3507831609,84.063858,3644.738929,1205.911576,1067.592624,713.191196,1171.474411,1212.815021,1153.184160,2866.745101,1022.086121,...,1238.024072,1258.666086,2314.361000,2314.067687,1210.127373,1206.020815,1205.492551,1205.405012,1025.566245,1032.429173
3507831510,90.422354,3651.097425,1212.270072,1073.951121,719.549692,1177.832907,1219.173518,1159.542656,2873.103598,1028.444618,...,1244.382568,1265.024582,2320.719497,2320.426184,1216.485869,1212.379312,1211.851048,1211.763508,1031.924741,1038.787670
6188134127,25.237636,3585.912707,1147.085354,1008.766402,654.364974,1112.648189,1153.988799,1094.357938,2807.918879,963.259899,...,1179.197850,1199.839864,2255.534778,2255.241465,1151.301151,1147.194593,1146.666329,1146.578790,966.740023,973.602951
8631201421,1070.956229,2782.475931,1300.323777,1162.004825,807.603397,1265.886612,1307.227222,1247.596361,2961.157302,1116.498322,...,1332.436272,1353.078287,2079.632570,2079.339257,1304.539574,1300.433016,1299.904752,1299.817212,1119.978446,1126.841374
8598305977,1085.282837,2796.802539,1314.650385,1176.331434,821.930005,1280.213220,1321.553831,1261.922969,2975.483911,1130.824931,...,1346.762881,1367.404895,2093.959178,2093.665866,1318.866182,1314.759625,1314.231361,1314.143821,1134.305054,1141.167983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651042474,292483.740922,295151.935823,291549.442212,291636.821761,291989.631562,291494.026219,291533.947794,291475.291145,293435.974238,291544.871690,...,291389.910453,291318.372106,293821.557894,293821.264582,291553.658009,291527.153588,291526.625324,291526.537784,291547.128617,291547.965567
3651042508,292475.590667,295143.785568,291541.291958,291628.671506,291981.481307,291485.875964,291525.797539,291467.140890,293427.823983,291536.721435,...,291381.760198,291310.221851,293813.407640,293813.114327,291545.507755,291519.003333,291518.475069,291518.387529,291538.978362,291539.815312
3651042501,292481.085783,295149.280684,291546.787073,291634.166622,291986.976423,291491.371079,291531.292654,291472.636005,293433.319099,291542.216551,...,291387.255313,291315.716967,293818.902755,293818.609443,291551.002870,291524.498448,291523.970184,291523.882645,291544.473478,291545.310427
3651042393,292467.555480,295135.750381,291533.256771,291620.636320,291973.446120,291477.840777,291517.762352,291459.105703,293419.788796,291528.686249,...,291373.725011,291302.186665,293805.372453,293805.079140,291537.472568,291510.968146,291510.439882,291510.352342,291530.943175,291531.780125


In [52]:
print(OD_ag.isna().sum())

6058226279    2
6029307183    2
4998093094    2
2201506815    2
4321656809    2
1697006012    2
1901689169    2
6032060028    2
6040927878    2
3449495495    2
3990543961    2
8972391475    2
3418418812    2
1983641803    2
6014451367    2
6027163276    2
2833577858    2
7321088861    2
6045659373    2
5528866190    2
6027939517    2
6031093257    2
1883155712    2
6026417971    2
6031245697    2
7493947593    2
1859359090    2
6053816462    2
6027530676    2
2988435647    2
7838754747    2
6053825179    2
2988435792    2
3036877534    2
6056472223    2
7460241128    2
6028595154    2
4053739078    2
4071261250    2
6014815775    2
6024430122    2
8229317581    2
6027834438    2
6024894515    2
6024894530    2
6024894513    2
8169321141    2
8200304626    2
1968458114    2
9359670710    2
6021177135    2
6027615161    2
6027276892    2
6041228287    2
5536661253    2
7357630367    2
8178147277    2
6026834850    2
dtype: int64


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [53]:
del gTime, ag_snap, hamlet_snap, HDurban_snap, OD, OD_df, OD_min
print(dir())

['GISFolder', 'GeoDataFrame', 'In', 'LineString', 'MultiLineString', 'OD_ag', 'OD_hamlet', 'Out', 'Point', 'Polygon', '_', '_1', '_14', '_23', '_25', '_35', '_39', '_40', '_46', '_47', '_49', '_51', '_7', '_8', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i4', '_i40', '_i41', '_i42', '_i43', '_i44', '_i45', '_i46', '_i47', '_i48', '_i49', '_i5', '_i50', '_i51', '_i52', '_i53', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'box', 'dests', 'exit', 'fail_value', 'fiona', 'gdal', 'get_ipython', 'glob', 'gn', 'gpd', 'importlib', 'linemerge', 'list_ag', 'list_hamlet', 'loads', 'losm', 'np', 'nx', 'origins', 'originslist', 'os', 'out_pth', 'ox

In [54]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
OD_hamlet = os.path.join(out_pth, "OD_hamlet_post-project_flood50.csv")
OD_hamlet = pd.read_csv(OD_hamlet)

In [55]:
OD_ag = os.path.join(out_pth, "OD_ag_post-project_flood50.csv")
OD_ag = pd.read_csv(OD_ag)

In [56]:
OD_ag.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 

In [57]:
OD_hamlet.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) 

In [58]:
OD_ag

,NN,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,3507831609,84.063858,3644.738929,1205.911576,1067.592624,713.191196,1171.474411,1212.815021,1153.184160,2866.745101,...,1238.024072,1258.666086,2314.361000,2314.067687,1210.127373,1206.020815,1205.492551,1205.405012,1025.566245,1032.429173
1,3507831510,90.422354,3651.097425,1212.270072,1073.951121,719.549692,1177.832907,1219.173518,1159.542656,2873.103598,...,1244.382568,1265.024582,2320.719497,2320.426184,1216.485869,1212.379312,1211.851048,1211.763508,1031.924741,1038.787670
2,6188134127,25.237636,3585.912707,1147.085354,1008.766402,654.364974,1112.648189,1153.988799,1094.357938,2807.918879,...,1179.197850,1199.839864,2255.534778,2255.241465,1151.301151,1147.194593,1146.666329,1146.578790,966.740023,973.602951
3,8631201421,1070.956229,2782.475931,1300.323777,1162.004825,807.603397,1265.886612,1307.227222,1247.596361,2961.157302,...,1332.436272,1353.078287,2079.632570,2079.339257,1304.539574,1300.433016,1299.904752,1299.817212,1119.978446,1126.841374
4,8598305977,1085.282837,2796.802539,1314.650385,1176.331434,821.930005,1280.213220,1321.553831,1261.922969,2975.483911,...,1346.762881,1367.404895,2093.959178,2093.665866,1318.866182,1314.759625,1314.231361,1314.143821,1134.305054,1141.167983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967547,3651042474,292483.740922,295151.935823,291549.442212,291636.821761,291989.631562,291494.026219,291533.947794,291475.291145,293435.974238,...,291389.910453,291318.372106,293821.557894,293821.264582,291553.658009,291527.153588,291526.625324,291526.537784,291547.128617,291547.965567
967548,3651042508,292475.590667,295143.785568,291541.291958,291628.671506,291981.481307,291485.875964,291525.797539,291467.140890,293427.823983,...,291381.760198,291310.221851,293813.407640,293813.114327,291545.507755,291519.003333,291518.475069,291518.387529,291538.978362,291539.815312
967549,3651042501,292481.085783,295149.280684,291546.787073,291634.166622,291986.976423,291491.371079,291531.292654,291472.636005,293433.319099,...,291387.255313,291315.716967,293818.902755,293818.609443,291551.002870,291524.498448,291523.970184,291523.882645,291544.473478,291545.310427
967550,3651042393,292467.555480,295135.750381,291533.256771,291620.636320,291973.446120,291477.840777,291517.762352,291459.105703,293419.788796,...,291373.725011,291302.186665,293805.372453,293805.079140,291537.472568,291510.968146,291510.439882,291510.352342,291530.943175,291531.780125


In [59]:
OD_hamlet

,NN,6058226279,6029307183,4998093094,2201506815,4321656809,1697006012,1901689169,6032060028,6040927878,...,1968458114,9359670710,6021177135,6027615161,6027276892,6041228287,5536661253,7357630367,8178147277,6026834850
0,7761872870,7555.041976,11114.143037,8675.315684,8536.996732,8182.595304,8640.878519,8682.219129,8622.588267,10336.149209,...,8707.428179,8728.070194,9783.765108,9783.471795,8679.531481,8675.424923,8674.896659,8674.809119,8494.970352,8501.833281
1,7761872869,7555.023417,11114.124477,8675.297124,8536.978173,8182.576744,8640.859959,8682.200570,8622.569708,10336.130649,...,8707.409620,8728.051634,9783.746548,9783.453236,8679.512921,8675.406364,8674.878100,8674.790560,8494.951793,8501.814721
2,6442044321,7554.013585,11113.114645,8674.287292,8535.968341,8181.566912,8639.850127,8681.190738,8621.559876,10335.120817,...,8706.399788,8727.041802,9782.736716,9782.443404,8678.503089,8674.396532,8673.868268,8673.780728,8493.941961,8500.804889
3,2142496418,7555.467107,11114.568167,8675.740814,8537.421863,8183.020434,8641.303649,8682.644260,8623.013398,10336.574339,...,8707.853310,8728.495324,9784.190238,9783.896926,8679.956611,8675.850054,8675.321790,8675.234250,8495.395483,8502.258411
4,2142496429,7555.849481,11114.950541,8676.123188,8537.804237,8183.402808,8641.686023,8683.026634,8623.395772,10336.956713,...,8708.235684,8728.877698,9784.572612,9784.279300,8680.338985,8676.232428,8675.704164,8675.616624,8495.777857,8502.640785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74657,9208004175,402592.030991,405260.225892,401657.732281,401745.111830,402097.921631,401602.316288,401642.237863,401583.581214,403544.264307,...,401498.200522,401426.662175,403929.847963,403929.554651,401661.948078,401635.443657,401634.915393,401634.827853,401655.418686,401656.255636
74658,3884280043,1900.785584,4568.980485,1151.720971,1053.866424,1406.676224,1114.461377,1154.382952,1095.693230,2853.018900,...,1001.608998,1075.735769,3238.602557,3238.309244,1155.936768,1147.588746,1147.060482,1146.972943,971.630082,964.146135
74659,3884280001,1893.923357,4562.118258,1144.858744,1047.004197,1399.813997,1107.599150,1147.520725,1088.831003,2846.156673,...,994.746771,1068.873542,3231.740330,3231.447017,1149.074541,1140.726519,1140.198255,1140.110716,964.767855,957.283908
74660,8592243457,1869.716935,4537.911836,1120.652322,1022.797774,1375.607575,1083.392728,1123.314303,1064.624581,2821.950251,...,970.540349,1044.667120,3207.533907,3207.240595,1124.868119,1116.520097,1115.991833,1115.904293,940.561432,933.077486


#### Find first, second, and third nearest destination for each origin node. 

In [60]:
fail_value = 999999999

In [61]:
# Nearest
OD_ag["ag_HD1"] = 0
sub = OD_ag.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_ag["ag_HD1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
ag1 = OD_ag[['NN', 'ag_HD1']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_ag["ag_HD2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_ag["ag_HD2"] = Dsub.min(axis=1) 
ag2 = OD_ag.loc[:,['NN', 'ag_HD2']] 


# Third nearest
dupes = OD_ag.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_ag.where(~dupes, fail_value)

OD_ag["ag_HD3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_ag["ag_HD3"] = Dsub.min(axis=1)
ag3 = OD_ag.loc[:,['NN', 'ag_HD3']]

# Combine and write to file
ag_all = OD_ag.loc[:,['NN', 'ag_HD1', 'ag_HD2', 'ag_HD3']]
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_drive_post-project_flood50.csv'))
ag_all.head()

,NN,ag_HD1,ag_HD2,ag_HD3
0,3507831609,82.363044,84.063858,687.136217
1,3507831510,88.721541,90.422354,693.494714
2,6188134127,23.536822,25.237636,628.309995
3,8631201421,615.678582,807.603397,1070.956229
4,8598305977,630.005190,821.930005,1085.282837


In [62]:
# Nearest
OD_hamlet["ha_HD1"] = 0
sub = OD_hamlet.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_hamlet["ha_HD1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
hamlet1 = OD_hamlet[['NN', 'ha_HD1']] # Remove unnecessary OD values.


# Second nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.
OD_hamlet["ha_HD2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_hamlet["ha_HD2"] = Dsub.min(axis=1) 
hamlet2 = OD_hamlet.loc[:,['NN', 'ha_HD2']] 


# Third nearest
dupes = OD_hamlet.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_hamlet.where(~dupes, fail_value)
OD_hamlet["ha_HD3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_hamlet["ha_HD3"] = Dsub.min(axis=1)
hamlet3 = OD_hamlet.loc[:,['NN', 'ha_HD3']]


# Combine and write to file
hamlet_all = OD_hamlet.loc[:,['NN', 'ha_HD1', 'ha_HD2', 'ha_HD3']]
hamlet_all.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_drive_post-project_flood50.csv'))
hamlet_all.head()

,NN,ha_HD1,ha_HD2,ha_HD3
0,7761872870,7555.041976,7558.410156,8156.540325
1,7761872869,7555.023417,7558.391596,8156.521766
2,6442044321,7554.013585,7557.381764,8155.511934
3,2142496418,7555.467107,7558.835286,8156.965456
4,2142496429,7555.849481,7559.217660,8157.347830


### End of script. Load into QGIS or Arc and visualize at 10 min intervals. 
QML file for symbology in QGIS:
R:\GEOGlobal\Design\symb_traveltimes_10min.qml